# Import Cool Stuff

In [1]:
from __future__ import print_function

from collections import defaultdict, deque
import datetime
import pickle
import time
import torch.distributed as dist
import errno
import math
import collections
import os
import numpy as np
import torch
import torch.utils.data
from skimage.measure import label
from PIL import Image, ImageFile
import pandas as pd
from tqdm import tqdm
from torchvision import transforms
import torchvision
import random
import sys
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor

ImageFile.LOAD_TRUNCATED_IMAGES = True


ModuleNotFoundError: No module named 'skimage'

In [ ]:
!git clone https://github.com/pytorch/vision.git
!cp -f ./vision/references/detection/transforms.py ./ 
!cp -f ./vision/references/detection/utils.py ./ 
!rm -r ./vision  

# Utility Functions (hidden)

In [ ]:
class SmoothedValue(object):
    """Track a series of values and provide access to smoothed values over a
    window or the global series average.
    """

    def __init__(self, window_size=20, fmt=None):
        if fmt is None:
            fmt = "{median:.4f} ({global_avg:.4f})"
        self.deque = deque(maxlen=window_size)
        self.total = 0.0
        self.count = 0
        self.fmt = fmt

    def update(self, value, n=1):
        self.deque.append(value)
        self.count += n
        self.total += value * n

    def synchronize_between_processes(self):
        """
        Warning: does not synchronize the deque!
        """
        if not is_dist_avail_and_initialized():
            return
        t = torch.tensor([self.count, self.total], dtype=torch.float64, device='cuda')
        dist.barrier()
        dist.all_reduce(t)
        t = t.tolist()
        self.count = int(t[0])
        self.total = t[1]

    @property
    def median(self):
        d = torch.tensor(list(self.deque))
        return d.median().item()

    @property
    def avg(self):
        d = torch.tensor(list(self.deque), dtype=torch.float32)
        return d.mean().item()

    @property
    def global_avg(self):
        return self.total / self.count

    @property
    def max(self):
        return max(self.deque)

    @property
    def value(self):
        return self.deque[-1]

    def __str__(self):
        return self.fmt.format(
            median=self.median,
            avg=self.avg,
            global_avg=self.global_avg,
            max=self.max,
            value=self.value)


def all_gather(data):
    """
    Run all_gather on arbitrary picklable data (not necessarily tensors)
    Args:
        data: any picklable object
    Returns:
        list[data]: list of data gathered from each rank
    """
    world_size = get_world_size()
    if world_size == 1:
        return [data]

    # serialized to a Tensor
    buffer = pickle.dumps(data)
    storage = torch.ByteStorage.from_buffer(buffer)
    tensor = torch.ByteTensor(storage).to("cuda")

    # obtain Tensor size of each rank
    local_size = torch.tensor([tensor.numel()], device="cuda")
    size_list = [torch.tensor([0], device="cuda") for _ in range(world_size)]
    dist.all_gather(size_list, local_size)
    size_list = [int(size.item()) for size in size_list]
    max_size = max(size_list)

    # receiving Tensor from all ranks
    # we pad the tensor because torch all_gather does not support
    # gathering tensors of different shapes
    tensor_list = []
    for _ in size_list:
        tensor_list.append(torch.empty((max_size,), dtype=torch.uint8, device="cuda"))
    if local_size != max_size:
        padding = torch.empty(size=(max_size - local_size,), dtype=torch.uint8, device="cuda")
        tensor = torch.cat((tensor, padding), dim=0)
    dist.all_gather(tensor_list, tensor)

    data_list = []
    for size, tensor in zip(size_list, tensor_list):
        buffer = tensor.cpu().numpy().tobytes()[:size]
        data_list.append(pickle.loads(buffer))

    return data_list


def reduce_dict(input_dict, average=True):
    """
    Args:
        input_dict (dict): all the values will be reduced
        average (bool): whether to do average or sum
    Reduce the values in the dictionary from all processes so that all processes
    have the averaged results. Returns a dict with the same fields as
    input_dict, after reduction.
    """
    world_size = get_world_size()
    if world_size < 2:
        return input_dict
    with torch.no_grad():
        names = []
        values = []
        # sort the keys so that they are consistent across processes
        for k in sorted(input_dict.keys()):
            names.append(k)
            values.append(input_dict[k])
        values = torch.stack(values, dim=0)
        dist.all_reduce(values)
        if average:
            values /= world_size
        reduced_dict = {k: v for k, v in zip(names, values)}
    return reduced_dict


class MetricLogger(object):
    def __init__(self, delimiter="\t"):
        self.meters = defaultdict(SmoothedValue)
        self.delimiter = delimiter

    def update(self, **kwargs):
        for k, v in kwargs.items():
            if isinstance(v, torch.Tensor):
                v = v.item()
            assert isinstance(v, (float, int))
            self.meters[k].update(v)

    def __getattr__(self, attr):
        if attr in self.meters:
            return self.meters[attr]
        if attr in self.__dict__:
            return self.__dict__[attr]
        raise AttributeError("'{}' object has no attribute '{}'".format(
            type(self).__name__, attr))

    def __str__(self):
        loss_str = []
        for name, meter in self.meters.items():
            loss_str.append(
                "{}: {}".format(name, str(meter))
            )
        return self.delimiter.join(loss_str)

    def synchronize_between_processes(self):
        for meter in self.meters.values():
            meter.synchronize_between_processes()

    def add_meter(self, name, meter):
        self.meters[name] = meter

    def log_every(self, iterable, print_freq, header=None):
        i = 0
        if not header:
            header = ''
        start_time = time.time()
        end = time.time()
        iter_time = SmoothedValue(fmt='{avg:.4f}')
        data_time = SmoothedValue(fmt='{avg:.4f}')
        space_fmt = ':' + str(len(str(len(iterable)))) + 'd'
        log_msg = self.delimiter.join([
            header,
            '[{0' + space_fmt + '}/{1}]',
            'eta: {eta}',
            '{meters}',
            'time: {time}',
            'data: {data}',
            'max mem: {memory:.0f}'
        ])
        MB = 1024.0 * 1024.0
        for obj in iterable:
            data_time.update(time.time() - end)
            yield obj
            iter_time.update(time.time() - end)
            if i % print_freq == 0 or i == len(iterable) - 1:
                eta_seconds = iter_time.global_avg * (len(iterable) - i)
                eta_string = str(datetime.timedelta(seconds=int(eta_seconds)))
                print(log_msg.format(
                    i, len(iterable), eta=eta_string,
                    meters=str(self),
                    time=str(iter_time), data=str(data_time),
                    memory=torch.cuda.max_memory_allocated() / MB))
            i += 1
            end = time.time()
        total_time = time.time() - start_time
        total_time_str = str(datetime.timedelta(seconds=int(total_time)))
        print('{} Total time: {} ({:.4f} s / it)'.format(
            header, total_time_str, total_time / len(iterable)))


def collate_fn(batch):
    return tuple(zip(*batch))


def warmup_lr_scheduler(optimizer, warmup_iters, warmup_factor):

    def f(x):
        if x >= warmup_iters:
            return 1
        alpha = float(x) / warmup_iters
        return warmup_factor * (1 - alpha) + alpha

    return torch.optim.lr_scheduler.LambdaLR(optimizer, f)


def mkdir(path):
    try:
        os.makedirs(path)
    except OSError as e:
        if e.errno != errno.EEXIST:
            raise


def setup_for_distributed(is_master):
    """
    This function disables printing when not in master process
    """
    import builtins as __builtin__
    builtin_print = __builtin__.print

    def print(*args, **kwargs):
        force = kwargs.pop('force', False)
        if is_master or force:
            builtin_print(*args, **kwargs)

    __builtin__.print = print


def is_dist_avail_and_initialized():
    if not dist.is_available():
        return False
    if not dist.is_initialized():
        return False
    return True


def get_world_size():
    if not is_dist_avail_and_initialized():
        return 1
    return dist.get_world_size()


def get_rank():
    if not is_dist_avail_and_initialized():
        return 0
    return dist.get_rank()


def is_main_process():
    return get_rank() == 0


def save_on_master(*args, **kwargs):
    if is_main_process():
        torch.save(*args, **kwargs)


def init_distributed_mode(args):
    if 'RANK' in os.environ and 'WORLD_SIZE' in os.environ:
        args.rank = int(os.environ["RANK"])
        args.world_size = int(os.environ['WORLD_SIZE'])
        args.gpu = int(os.environ['LOCAL_RANK'])
    elif 'SLURM_PROCID' in os.environ:
        args.rank = int(os.environ['SLURM_PROCID'])
        args.gpu = args.rank % torch.cuda.device_count()
    else:
        print('Not using distributed mode')
        args.distributed = False
        return

    args.distributed = True

    torch.cuda.set_device(args.gpu)
    args.dist_backend = 'nccl'
    print('| distributed init (rank {}): {}'.format(
        args.rank, args.dist_url), flush=True)
    torch.distributed.init_process_group(backend=args.dist_backend, init_method=args.dist_url,
                                         world_size=args.world_size, rank=args.rank)
    torch.distributed.barrier()
    setup_for_distributed(args.rank == 0)

# Cancer Dataset Class

In [ ]:
class CancerDataset(torch.utils.data.Dataset):
        
    def __init__(self, images, masks, valid_ids, transforms=None): #定义的全局变量必须加self.
        self.images, self.masks = images, masks
        self.valid_ids = valid_ids
        self.transforms = transforms                               #transform用于数据增强
 
    def __getitem__(self, idx):
        idx = self.valid_ids[idx]
        img = self.images[idx,:,:,:]
        # print('masks shape:', img.shape)
        mask = self.masks[idx,:,:,0]
        obj_ids = np.unique(mask)
        # print('obj ids:', obj_ids)
        # 去除切片维度，也就是[0]
        obj_ids = obj_ids[1:]
        masks = mask == obj_ids[:, None, None]              #把每张图都拿出来放到masks里,拓展维度
        # print('masks shape:', masks.shape)
        
        #给每个mask加框
        num_objs = len(obj_ids)
        boxes = []
        for i in range(num_objs):
            pos = np.where(masks[i])
            xmin = np.min(pos[1])                           #列是1，行是0
            xmax = np.max(pos[1])
            ymin = np.min(pos[0])
            ymax = np.max(pos[0])
#             boxes.append([xmin, ymin, xmax-xmin, ymax-ymin])          #append：往列表里加元素
            if (xmax-xmin) > 3 and (ymax-ymin) > 3:
                boxes.append([xmin, ymin, xmax, ymax])          #append：往列表里加元素
 
#         boxes = torch.as_tensor(boxes, dtype=torch.float32)
        boxes = torch.as_tensor(boxes)
        labels = torch.ones((num_objs,), dtype=torch.int64) -1
        masks = torch.as_tensor(masks, dtype=torch.uint8)
        
        image_id = torch.tensor([idx])
    
        target = {}
        target["boxes"] = boxes
        target["masks"] = masks
        target["image_id"] = image_id
        target["labels"] = labels
        
        if self.transforms is not None:
            img, target = self.transforms(img, target)
 
 
        return img, target
 
 
    def __len__(self):
        return self.valid_ids.shape[0]
#         return self.images.shape[0]                       #返回值不能是numpy数组，为切片数

In [ ]:
import transforms as T
import utils
 
def get_transform(train):
    transforms = []
    # 把PIL图转为张量
    transforms.append(T.ToTensor())
    if train:
        transforms.append(T.RandomHorizontalFlip(0.5))     #随机水平翻转,翻一倍
    return T.Compose(transforms)

In [ ]:
import os
import numpy as np

root = '../input/cancer-inst-segmentation-and-classification'
img_path = os.path.join(root, 'Images/images.npy')         #root是mask和image公共的路径部分
images = np.load(img_path,mmap_mode='r')
mask_path = os.path.join(root, 'Masks/masks.npy')
masks = np.load(mask_path,mmap_mode='r')


def filter_masks(masks):
    valid_idxs = []
    for idx in range(masks.shape[0]):
        m = masks[idx,:,:,0]
        obj_ids = np.unique(m)
        if len(obj_ids) == 1:
            continue
        else:
            obj_ids = obj_ids[1:]
            ms = m == obj_ids[:, None, None]              #把每张图都拿出来放到masks里,拓展维度
            #给每个mask加框
            num_objs = len(obj_ids)
            boxes = []
            for i in range(num_objs):
                pos = np.where(ms[i])
                xmin = np.min(pos[1])                           #列是1，行是0
                xmax = np.max(pos[1])
                ymin = np.min(pos[0])
                ymax = np.max(pos[0])
    #             boxes.append([xmin, ymin, xmax-xmin, ymax-ymin])          #append：往列表里加元素
                if (xmax-xmin) > 3 and (ymax-ymin) > 3:
                    boxes.append([xmin, ymin, xmax, ymax])
            if len(boxes) > 0:
                valid_idxs.append(idx)  
    return np.array(valid_idxs)

valid_idxs = filter_masks(masks)
# images_filter, masks_filter = images[valid_idxs], masks[valid_idxs]
int(len(valid_idxs)*0.8)  # 1459

# 计算有效图片在原序列中的位置
trn_id, val_id = int(len(valid_idxs)*0.8), int(len(valid_idxs)*0.9)
print('filter ids:',trn_id,val_id)
print('org ids:', valid_idxs[trn_id],valid_idxs[val_id])
print('trainset len:',valid_idxs[:trn_id].shape)
print('valset len:',valid_idxs[trn_id:val_id].shape)
print('testset len:',valid_idxs[val_id:].shape)

images_split = np.split(images,[valid_idxs[trn_id],valid_idxs[val_id]])
print(images_split[0].shape)

masks_split = np.split(masks,[valid_idxs[trn_id],valid_idxs[val_id]])
print(masks_split[1].shape)


tr_idxs = valid_idxs[:trn_id]
val_idxs = valid_idxs[trn_id:val_id] - valid_idxs[trn_id]
ts_idxs = valid_idxs[val_id:] - valid_idxs[val_id]

trainset = CancerDataset(images_split[0], masks_split[0], tr_idxs, get_transform(train=True))
valset = CancerDataset(images_split[1], masks_split[1], val_idxs, get_transform(train=False))
testset = CancerDataset(images_split[2], masks_split[2], ts_idxs, get_transform(train=False))
print('trainset len:',len(trainset),'valset len:',len(valset),'testset len:',len(testset))

# Define Dataloader

In [ ]:
data_loader = torch.utils.data.DataLoader(
    trainset, batch_size=2, shuffle=True, num_workers=0,
    collate_fn=lambda x: tuple(zip(*x)))

data_loader_val = torch.utils.data.DataLoader(
    valset, batch_size=2, shuffle=False, num_workers=0,
    collate_fn=lambda x: tuple(zip(*x)))

# Create Mask-RCNN Model

In [ ]:
# create mask rcnn model
num_classes = 1
device = torch.device('cuda:0')


model_ft = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)
print('model_ft: ', model_ft)
in_features = model_ft.roi_heads.box_predictor.cls_score.in_features
print('in_features: ', in_features)

model_ft.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
in_features_mask = model_ft.roi_heads.mask_predictor.conv5_mask.in_channels
print('in_features_mask: ', in_features_mask)
hidden_layer = 256
model_ft.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask, hidden_layer, num_classes)
model_ft.to(device)
    
    

for param in model_ft.parameters():
    param.requires_grad = True

# Training Function

In [ ]:
def train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq):
    model.train()
    metric_logger = MetricLogger(delimiter="  ")
    metric_logger.add_meter('lr', SmoothedValue(window_size=1, fmt='{value:.6f}'))
    header = 'Epoch: [{}]'.format(epoch)

    lr_scheduler = None
    if epoch == 0:
        warmup_factor = 1. / 1000
        warmup_iters = min(1000, len(data_loader) - 1)

        lr_scheduler = utils.warmup_lr_scheduler(optimizer, warmup_iters, warmup_factor)
        #加了utils
        
    for images, targets in metric_logger.log_every(data_loader, print_freq, header):
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        images = list(image.float() for image in images)   #加了这句
        
        loss_dict = model_ft(images, targets)
        print('loss_dict:',loss_dict)

        losses = sum(loss for loss in loss_dict.values())

        # reduce losses over all GPUs for logging purposes
        loss_dict_reduced = utils.reduce_dict(loss_dict)    #加了utils
        losses_reduced = sum(loss for loss in loss_dict_reduced.values())
        loss_value = losses_reduced.item()        #加了这句 
                          

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

        if lr_scheduler is not None:
            lr_scheduler.step()

        metric_logger.update(loss=losses_reduced, **loss_dict_reduced)
        metric_logger.update(lr=optimizer.param_groups[0]["lr"])
        
    

# Define Training Parameters

In [ ]:
params = [p for p in model_ft.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.001, momentum=0.9, weight_decay=0.0005)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                               step_size=5,
                                               gamma=0.1)

# Train Model

In [ ]:
num_epochs = 20
for epoch in range(num_epochs):
    train_one_epoch(model_ft, optimizer, data_loader, device, epoch, print_freq=100)
    lr_scheduler.step()

# Convert Model to Evaluation Mode

In [ ]:
for param in model_ft.parameters():
    param.requires_grad = False

model_ft.eval()

# Get Test Data

In [ ]:
# # this part was taken from @raddar's kernel: https://www.kaggle.com/raddar/better-sample-submission
# masks_ = sample_df.groupby('ImageId')['ImageId'].count().reset_index(name='N')
# masks_ = masks_.loc[masks_.N > 1].ImageId.values
# ###
# sample_df = sample_df.drop_duplicates('ImageId', keep='last').reset_index(drop=True)

In [ ]:
# tt = transforms.ToTensor()
# sublist = []
# counter = 0
# threshold = 0.3
# for index, row in tqdm(sample_df.iterrows(), total=len(sample_df)):
#     image_id = row['ImageId']
#     if image_id in masks_:
#         img_path = os.path.join('../input/siim-png-images/input/test_png', image_id + '.png')

#         img = Image.open(img_path).convert("RGB")
#         width, height = img.size
#         img = img.resize((1024, 1024), resample=Image.BILINEAR)
#         img = tt(img)
#         result = model_ft([img.to(device)])[0]
#         if len(result["masks"]) > 0:
#             counter += 1
#             mask_added = 0
#             for ppx in range(len(result["masks"])):
#                 if result["scores"][ppx] >= threshold:
#                     mask_added += 1
#                     res = transforms.ToPILImage()(result["masks"][ppx].permute(1, 2, 0).cpu().numpy())
#                     res = np.asarray(res.resize((width, height), resample=Image.BILINEAR))
#                     res = (res[:, :] * 255. > 127).astype(np.uint8).T
#                     rle = mask_to_rle(res, width, height)
#                     sublist.append([image_id, rle])
#             if mask_added == 0:
#                 rle = " -1"
#                 sublist.append([image_id, rle])
#         else:
#             rle = " -1"
#             sublist.append([image_id, rle])
#     else:
#         rle = " -1"
#         sublist.append([image_id, rle])

# submission_df = pd.DataFrame(sublist, columns=sample_df.columns.values)
# submission_df.to_csv("submission.csv", index=False)
# print(counter)